Data from: https://www.sciencedirect.com/science/article/pii/S1535610822002744#mmc3   
Pan-cancer proteomic map of 949 human cell lines


In [1]:
import mlmarker
from mlmarker.model import MLMarker

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
import umap
from sklearn.decomposition import PCA


In [3]:
df = pd.read_excel('data.xlsx', sheet_name='Prot matrix excl single-peptide', skiprows=1, index_col=0, nrows=100)
df = df.transpose().reset_index()
df

Project_Identifier,index,SIDM00018;K052,SIDM00023;TE-12,SIDM00040;TMK-1,SIDM00041;STS-0421,SIDM00042;PL4,SIDM00043;PCI-4B,SIDM00044;PCI-30,SIDM00045;HSC-39,SIDM00046;H3255,...,SIDM00176;CHSA8926,SIDM00179;MZ2-MEL,SIDM00180;LB996-RCC,SIDM00181;LB831-BLC,SIDM00182;LB771-HNC,SIDM00183;LB647-SCLC,SIDM00184;LB373-MEL-D,SIDM00185;LB2518-MEL,SIDM00186;LB2241-RCC,SIDM00187;LB1047-RCC
0,P37108;SRP14_HUMAN,7.10955,6.82802,7.01426,5.28591,5.70786,6.73965,6.04591,6.20582,6.53547,...,5.56615,6.10347,6.07923,5.23132,6.33619,7.37676,6.09422,5.83401,5.73337,6.89669
1,Q96JP5;ZFP91_HUMAN,3.41494,4.14346,4.19987,3.35789,NaN,4.50199,3.69356,2.88118,NaN,...,NaN,3.60707,2.80437,2.22452,3.64149,3.79681,3.94879,3.48848,NaN,3.33413
2,Q9Y4H2;IRS2_HUMAN,NaN,2.23781,2.44055,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.06960,1.52496,NaN,3.34668,1.53053,2.82872,3.96016,2.85972,2.72827
3,P36578;RL4_HUMAN,7.86661,7.62878,8.12459,7.97268,6.22574,7.47364,7.07092,8.25336,6.47861,...,7.16703,7.80277,7.91449,7.52710,7.64464,7.17964,8.00794,8.00271,7.05454,7.64372
4,Q6SPF0;SAMD1_HUMAN,3.89547,3.19811,NaN,NaN,NaN,NaN,3.49594,3.35439,NaN,...,NaN,1.98149,NaN,3.22246,4.20897,4.70107,2.19203,3.43305,NaN,3.21860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6687,Q7Z3B1;NEGR1_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.69603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6688,O60669;MOT2_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,2.16952,NaN,NaN,...,NaN,NaN,NaN,1.97720,NaN,NaN,NaN,NaN,NaN,NaN
6689,Q13571;LAPM5_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6690,Q96JM2;ZN462_HUMAN,NaN,NaN,NaN,NaN,NaN,3.39328,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.84846,3.77295,NaN,NaN,NaN,NaN


In [4]:
df.set_index('index', inplace=True)
df.reset_index(inplace=True)

# Penalty factor determining

In [5]:
metadata = pd.read_excel('metadata.xlsx', sheet_name = 'Cell line level sample info', skiprows=1)
#where metadata.Tissue_type is 'Haematopietic and Lymphoid', replace with the content from Haem_lineage column except if that is NaN
metadata['Tissue_type'] = metadata.apply(lambda x: x['Haem_lineage'] if x['Tissue_type'] == 'Haematopoietic and Lymphoid' else x['Tissue_type'], axis=1)
tissue_type_map = metadata.set_index('Cell_line')[['Tissue_type']].to_dict()['Tissue_type']

metadata_to_mlmarker = {
    'Haematopoietic and Lymphoid': ['Lymph node', 'T-cells', 'B-cells', 'NK-cells'],
    'Lung': [],  # No exact match, left empty
    'Stomach': ['Colon', 'Small intestine'],
    'Breast': [],  # No match in MLMarker_list
    'Bone': [],  # No match in MLMarker_list
    'Ovary': ['Ovary'],
    'Skin': [],  # No match in MLMarker_list
    'Head and Neck': ['Parotid gland'],
    'Endometrium': [],  # No match in MLMarker_list
    'Adrenal Gland': ['Adrenal gland'],
    'Central Nervous System': ['Brain'],
    'Pancreas': [],  # No match in MLMarker_list
    'Prostate': ['Prostate'],
    'Peripheral Nervous System': ['Brain'],
    'Other tissue': [],  # Generic, no direct mapping
    'Large Intestine': ['Colon', "Small intestine"],
    'Cervix': [],  # No match in MLMarker_list
    'Kidney': [],  # No match in MLMarker_list
    'Liver': ['Liver'],
    'Soft Tissue': ['Adipose tissue'],
    'Biliary Tract': ['Gall bladder'],
    'Bladder': ['Urinary bladder'],
    'Esophagus': ['Esophagus'],
    'Testis': ['Testis'],
    'Thyroid': [],  # No match in MLMarker_list
    'Vulva': [],  # No match in MLMarker_list
    'Small Intestine': ['Small intestine', "Colon"],
    'Placenta': ['Placenta'], 
    'B lymphoid': ['B-cells'], 'Myeloid' : ['Lymph node'],'T lymphoid':['T-cells'], 'NK lymphoid': ['NK-cells']
}

haem_lineage_to_mlmarker = {}

In [24]:
tissues_to_keep = []
cell_lines_to_keep = []
for k,v in metadata_to_mlmarker.items():
    if len(v) != 0:
        tissues_to_keep.append(k)
for k,v in tissue_type_map.items():
    if v in tissues_to_keep:
        cell_lines_to_keep.append(k)
metadata_matched = metadata[metadata['Cell_line'].isin(cell_lines_to_keep)]
projects_to_keep = metadata_matched['Project_Identifier'].unique()
df_matched = df.set_index('index')[df.columns[df.columns.isin(projects_to_keep)]].reset_index()
#column is in metadata_matched['Project_Identifier']

In [6]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from itertools import product
from mlmarker.model import MLMarker
from tqdm import tqdm

# Define penalty formula
def compute_penalty_factor(missingness_percentage, k, m0):
    return 2 / (1 + np.exp(-k * (missingness_percentage - m0)))

# Generate sigmoid curves for visualization
def visualize_sigmoid_curves(k_values, m0_values):
    missingness_range = np.linspace(0, 1, 100)
    for k, m0 in product(k_values, m0_values):
        penalties = compute_penalty_factor(missingness_range, k, m0)
        fig = px.line(x=missingness_range, y=penalties, 
                      title=f"Penalty Curve (k={k}, m0={m0})",
                      labels={"x": "Missingness", "y": "Penalty Factor"})
        fig.show()

# Predict for each sample and compute accuracy
def predict_and_evaluate(df, k_values, m0_values, tissue_type_map, metadata_to_mlmarker):
    results = []
    param_combinations = list(product(k_values, m0_values))
    for k, m0 in tqdm(param_combinations, desc="Parameter combinations", leave=True):
        prediction_dict = {}
        correct_predictions = 0
        total_predictions = 0
        correct_predictions_top3 = 0
        total_predictions_top3 = 0

        for c in tqdm(df.columns[1:], desc=f"Processing samples (k={k}, m0={m0})", leave=True):
            print(c)
            # Preprocess data
            subset = df[["index", c]].dropna()
            scaler = MinMaxScaler()
            subset[c] = scaler.fit_transform(subset[[c]])
            subset["index"] = subset["index"].apply(lambda x: x.split(";")[0])
            data = subset.pivot_table(columns="index", values=c, aggfunc="sum").fillna(0)
            
            # Compute missingness and penalty factor
            missingness = (data == 0).mean(axis=1).iloc[0]
            penalty_factor = compute_penalty_factor(missingness, k, m0)
            # Run MLMarker prediction
            test = MLMarker(data.iloc[0:1, :], binary=False)
            prediction = test.explainability.adjusted_absent_shap_values_df(penalty_factor=penalty_factor)
            prediction = prediction.sum(axis=1).sort_values(ascending=False)
            prediction_dict[c] = prediction
            
            # Evaluate prediction
            expected_tissue = tissue_type_map.get(c.split(';')[1], None)
            if not expected_tissue:
                continue  # Skip if no mapping exists for the column
            
            mlmarker_tissues = metadata_to_mlmarker.get(expected_tissue, [])
            top_prediction = prediction.index[0] if not prediction.empty else None
            if top_prediction in mlmarker_tissues:
                correct_predictions += 1
            total_predictions += 1

            #alsoo look at top 3 of predictions
            top3_predictions = prediction.index[:3] if not prediction.empty else None
            print(top3_predictions, mlmarker_tissues)
            if all(elem in top3_predictions for elem in mlmarker_tissues):
                correct_predictions_top3 += 1
            total_predictions_top3 += 1

        accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
        accuracy_top3 = correct_predictions_top3 / total_predictions_top3 if total_predictions_top3 > 0 else 0
        print(accuracy)
        results.append({"k": k, "m0": m0, "accuracy": accuracy, "accuracy_top3": accuracy_top3})
    return results

In [81]:
k_values = range(1,50,5) #steepness
m0_values = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95] #midpoint

# # Visualize sigmoid curves
# visualize_sigmoid_curves(k_values, m0_values)

# Predict and evaluate
results_df = predict_and_evaluate(df, k_values, m0_values, tissue_type_map, metadata_to_mlmarker)


Parameter combinations:   0%|          | 0/100 [00:00<?, ?it/s]

SIDM00018;K052
Features added: 2221, removed: 1527, remaining: 2163


Parameter combinations:   0%|          | 0/100 [00:01<?, ?it/s]

Index(['NK-cells', 'B-cells', 'Parotid gland'], dtype='object', name='tissue') ['Lymph node']


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
fig = px.scatter(results_df, x="k", y="m0", size="accuracy", color="accuracy",
                 title="Prediction Accuracy for Different Penalty Parameters",
                 labels={"k": "Steepness (k)", "m0": "Midpoint (m0)", "accuracy": "Accuracy"})
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Subplot visualization with accuracy in titles
def visualize_results_grid_with_accuracy(results_df, k_values, m0_values):
    # Create subplots grid
    fig = make_subplots(
        rows=len(k_values),
        cols=len(m0_values),
        shared_xaxes=True,
        shared_yaxes=True,
        subplot_titles=[
            f"k={k}, m0={m0}" for k in k_values for m0 in m0_values
        ],
        vertical_spacing=0.1,
        horizontal_spacing=0.05
    )

    for i, k in enumerate(k_values):
        for j, m0 in enumerate(m0_values):
            # Filter the data for the current combination of k and m0
            subset = results_df[(results_df["k"] == k) & (results_df["m0"] == m0)]

            if not subset.empty:
                # Compute mean accuracy for this combination
                mean_accuracy = subset["accuracy"].mean()
                
                # Add scatter plot for this combination
                fig.add_trace(
                    go.Scatter(
                        x=subset["sample"],  # Example x-axis (adjust as needed)
                        y=subset["accuracy"],  # Example y-axis
                        mode="lines+markers",
                        name=f"k={k}, m0={m0}",
                        marker=dict(size=8),
                        line=dict(width=2)
                    ),
                    row=i + 1,
                    col=j + 1
                )
                
                # Update subplot title with accuracy
                fig.layout.annotations[len(m0_values) * i + j].text = f"k={k}, m0={m0}<br>Accuracy={mean_accuracy:.2%}"

    # Update layout
    fig.update_layout(
        height=300 * len(k_values),  # Dynamic height for rows
        width=300 * len(m0_values),  # Dynamic width for columns
        title_text="Prediction Accuracy Across k and m0 Variations",
        showlegend=False
    )
    fig.update_xaxes(title_text="Sample Index", showgrid=True)
    fig.update_yaxes(title_text="Accuracy", range=[0, 1], showgrid=True)

    fig.show()

# Example usage
visualize_results_grid_with_accuracy(results_df, k_values, m0_values)


TypeError: list indices must be integers or slices, not str

In [53]:
results_df

[{'k': 1, 'm0': 0.5, 'accuracy': 0},
 {'k': 1, 'm0': 0.55, 'accuracy': 0},
 {'k': 50, 'm0': 0.5, 'accuracy': 0},
 {'k': 50, 'm0': 0.55, 'accuracy': 0}]

In [27]:
def linear_penalty(missingness, k, m0):
    return max(0, k * (missingness - m0))

def exponential_penalty(missingness, k, m0):
    return np.exp(k * (missingness - m0)) - 1

def sigmoid_penalty(missingness, k, m0):
    return 1 / (1 + np.exp(-k * (missingness - m0)))

def compute_penalty_factor(missingness, k, m0, penalty_type):
    if penalty_type == "linear":
        return linear_penalty(missingness, k, m0)
    elif penalty_type == "exponential":
        return exponential_penalty(missingness, k, m0)
    elif penalty_type == "sigmoid":
        return sigmoid_penalty(missingness, k, m0)
    else:
        raise ValueError(f"Unknown penalty type: {penalty_type}")


In [28]:
from tqdm import tqdm

def predict_and_evaluate(df, k_values, m0_values, tissue_type_map, metadata_to_mlmarker, penalty_types):
    results = []

    for penalty_type in penalty_types:
        print(f"Testing penalty type: {penalty_type}")
        for k, m0 in product(k_values, m0_values):
            print(f"Started on k = {k}, m0 = {m0}")
            prediction_dict = {}
            correct_predictions = 0
            total_predictions = 0
            correct_predictions_top3 = 0
            total_predictions_top3 = 0

            for c in tqdm(df.columns[1:15], desc=f"Samples for k={k}, m0={m0}"):
                # Preprocess data
                subset = df[["index", c]].dropna()
                scaler = MinMaxScaler()
                subset[c] = scaler.fit_transform(subset[[c]])
                subset["index"] = subset["index"].apply(lambda x: x.split(";")[0])
                data = subset.pivot_table(columns="index", values=c, aggfunc="sum").fillna(0)
                
                # Compute missingness and penalty factor
                missingness = (data == 0).mean(axis=1).iloc[0]
                penalty_factor = compute_penalty_factor(missingness, k, m0, penalty_type)
                
                # Run MLMarker prediction
                test = MLMarker(data.iloc[0:1, :], binary=False)
                prediction = test.explainability.adjusted_absent_shap_values_df(penalty_factor=penalty_factor)
                prediction = prediction.sum(axis=1).sort_values(ascending=False)
                prediction_dict[c] = prediction
                
                
                # Evaluate prediction
                expected_tissue = tissue_type_map.get(c.split(';')[1], None)
                if not expected_tissue:
                    continue  # Skip if no mapping exists for the column
                
                mlmarker_tissues = metadata_to_mlmarker.get(expected_tissue, [])
                top_prediction = prediction.index[0] if not prediction.empty else None
                if top_prediction in mlmarker_tissues:
                    correct_predictions += 1
                total_predictions += 1

                #alsoo look at top 3 of predictions
                top3_predictions = prediction.index[:3] if not prediction.empty else None
                print(top3_predictions, mlmarker_tissues)
                if all(elem in top3_predictions for elem in mlmarker_tissues):
                    correct_predictions_top3 += 1
                total_predictions_top3 += 1

            accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
            accuracy_top3 = correct_predictions_top3 / total_predictions_top3 if total_predictions_top3 > 0 else 0
        
            print(f"Accuracy for k={k}, m0={m0}, penalty={penalty_type}: {accuracy:.2f}")
            results.append({"k": k, "m0": m0, "penalty_type": penalty_type, "accuracy": accuracy, "accuracy_top3": accuracy_top3})
    
    return pd.DataFrame(results)


In [29]:
import plotly.express as px

def visualize_results(results_df):
    fig = px.scatter(
        results_df,
        x="k",
        y="m0",
        size="accuracy",
        color="penalty_type",
        facet_col="penalty_type",
        title="Accuracy Comparison Across Penalty Types",
        labels={"k": "Steepness (k)", "m0": "Midpoint (m0)", "accuracy": "Accuracy"}
    )
    fig.show()


In [31]:
k_values = range(1,50,5) #steepness
m0_values = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95] #midpoint
penalty_types = ["linear", "exponential", "sigmoid"]
# # Visualize sigmoid curves
# visualize_sigmoid_curves(k_values, m0_values)

results_df = predict_and_evaluate(df_matched, k_values, m0_values, tissue_type_map, metadata_to_mlmarker, penalty_types)

Testing penalty type: linear
Started on k = 1, m0 = 0.5


Samples for k=1, m0=0.5:   0%|          | 0/14 [00:00<?, ?it/s]

Features added: 2221, removed: 1527, remaining: 2163


Samples for k=1, m0=0.5:   0%|          | 0/14 [00:01<?, ?it/s]

Index(['B-cells', 'NK-cells', 'Monocytes'], dtype='object', name='tissue') ['Lymph node']


UnboundLocalError: local variable 'total_predictions_top3' referenced before assignment

# Match prediction to feature space

In [21]:
features_path = "/home/compomics/git/MLMarker/mlmarker/models/features_TP_full_92%_10exp_2024.txt"   
with open(features_path, 'r') as features_file:
    MLMarker_features = features_file.read().split(',\n')

In [22]:
sample_amount = {}
for i in df.columns:
    sample_amount[i] = len((df[i].dropna().index.str.split(';').str[0]).intersection(MLMarker_features))  

In [24]:
prediction_dict = {}  
df.reset_index(inplace=True)
#get the columns: "Protein.Ids", "	TU014944PAP_Slot1-12_1_3201.d	TU014945PAP_Slot1-"
for c in df.columns[1:]:
    subset = df[["index", c]]
    subset.dropna(inplace=True)
    #minmax scale the data in column c
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    subset[c] = scaler.fit_transform(subset[[c]])
    subset["index"] = subset["index"].apply(lambda x: x.split(";")[0])
    data = subset.pivot_table(columns='index', values=c, aggfunc='sum')
    data = data.fillna(0)
    test = MLMarker(data.iloc[0:1,:], binary = False)
    prediction = test.explainability.adjusted_absent_shap_values_df(penalty_factor=1.2)
    prediction = prediction.sum(axis=1).sort_values(ascending=False)
    prediction_dict[c] = prediction 

/tmp/ipykernel_66919/3367598466.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_66919/3367598466.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_66919/3367598466.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/compomics/miniconda3/envs/mlmarker/lib/python3.9/site-packages/sk

KeyError: 0

In [ ]:
prediction_df = pd.DataFrame(prediction_dict).fillna(0)
prediction_df.head()

In [ ]:
# prediction_df = pd.DataFrame(prediction_dict).fillna(0)
# prediction_df.to_csv('Cellline_predictions_normalized.csv')
prediction_df =  pd.read_csv('Cellline_predictions_normalized.csv', index_col=0)
prediction_df.head()

In [ ]:
#replace all negative values with zero
prediction_df[prediction_df < 0] = 0

In [ ]:
sns.clustermap(prediction_df, cmap='crest', xticklabels=False)

In [ ]:
prediction_df.loc['sample_amount'] = prediction_df.columns.map(sample_amount)
selected_tissues = ['Parotid gland', 'Brain', 'Pituitary gland', 'NK-cells', 'Liver', 'sample_amount']

selection = prediction_df[prediction_df.index.isin(selected_tissues)].T

# make four subplots
fig, axs = plt.subplots(3, 2, figsize=(10, 15))

for i, tissue in enumerate(selected_tissues):
    row = i // 2
    col = i % 2
    ax = axs[row, col]
    sns.regplot(x='sample_amount', y=tissue, data=selection, ax=ax, ci=None, scatter_kws={"color": "grey"}, line_kws={"color": "darkblue"})
    ax.set_xlabel('sample_amount')
    ax.set_ylabel(tissue)
    ax.set_ylim(bottom=-0.0005)  # Set the lower limit of the y-axis to zero
    ax.set_title(f'Number of Proteins vs {tissue}')

plt.tight_layout()
plt.show()


In [ ]:
metadata = pd.read_excel('metadata.xlsx', sheet_name = 'Cell line level sample info', skiprows=1)
#where metadata.Tissue_type is 'Haematopietic and Lymphoid', replace with the content from Haem_lineage column except if that is NaN
metadata['Tissue_type'] = metadata.apply(lambda x: x['Haem_lineage'] if x['Tissue_type'] == 'Haematopoietic and Lymphoid' else x['Tissue_type'], axis=1)

tissue_type_map = metadata.set_index('Cell_line')[['Tissue_type']].to_dict()['Tissue_type']
x = prediction_df.T.reset_index().rename(columns={'index': 'Project_Identifier'}).set_index('Project_Identifier')
df_prediction_full = x.merge(metadata, left_index=True, right_on='Project_Identifier')
df_prediction_full.set_index('Tissue_type', inplace=True)

In [ ]:
metadata.head()

In [ ]:
legend_labels

In [ ]:
import umap
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
import seaborn as sns
import matplotlib.patches as mpatches

# Perform PCA with 35 components
pca = PCA(n_components=35)
pca_components = pca.fit_transform(df_prediction_full.iloc[:, :35])

# Apply UMAP on the PCA components
reducer = umap.UMAP()
embedding = reducer.fit_transform(pca_components)
# Create a distinct color palette with 35 colors
palette = sns.color_palette("tab20", 20) + sns.color_palette("tab20b", 15)  # Combine palettes for 35 colors
cmap = ListedColormap(palette)


# Create scatter plot
unique_classes = df_prediction_full.index.unique()
color_mapping = {cls: i for i, cls in enumerate(unique_classes)}
colors = df_prediction_full.index.map(color_mapping).astype(int)
scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=colors, cmap=cmap)

# Add legend
legend_handles = [
    mpatches.Patch(color=palette[i], label=cls) for i, cls in enumerate(unique_classes)
]
plt.legend(
    handles=legend_handles,
    bbox_to_anchor=(1.05, 1),
    loc="upper left",
    title="Classes"
)

# Show plot
plt.show()


Is there a match btw Tissue_type and predicted tissue?

In [ ]:
metadata_to_mlmarker = {
    'Haematopoietic and Lymphoid': ['Lymph node', 'T-cells', 'B-cells', 'NK-cells'],
    'Lung': [],  # No exact match, left empty
    'Stomach': ['Colon', 'Small intestine'],
    'Breast': [],  # No match in MLMarker_list
    'Bone': [],  # No match in MLMarker_list
    'Ovary': ['Ovary'],
    'Skin': [],  # No match in MLMarker_list
    'Head and Neck': ['Parotid gland'],
    'Endometrium': [],  # No match in MLMarker_list
    'Adrenal Gland': ['Adrenal gland'],
    'Central Nervous System': ['Brain'],
    'Pancreas': [],  # No match in MLMarker_list
    'Prostate': ['Prostate'],
    'Peripheral Nervous System': ['Brain'],
    'Other tissue': [],  # Generic, no direct mapping
    'Large Intestine': ['Colon', "Small intestine"],
    'Cervix': [],  # No match in MLMarker_list
    'Kidney': [],  # No match in MLMarker_list
    'Liver': ['Liver'],
    'Soft Tissue': ['Adipose tissue'],
    'Biliary Tract': ['Gall bladder'],
    'Bladder': ['Urinary bladder'],
    'Esophagus': ['Esophagus'],
    'Testis': ['Testis'],
    'Thyroid': [],  # No match in MLMarker_list
    'Vulva': [],  # No match in MLMarker_list
    'Small Intestine': ['Small intestine', "Colon"],
    'Placenta': ['Placenta'], 
    'B lymphoid': ['B-cells'], 'Myeloid' : ['Lymph node'],'T lymphoid':['T-cells'], 'NK lymphoid': ['NK-cells']
}

haem_lineage_to_mlmarker = {}

In [ ]:
def check_top_n(row, n):
    tissue_type = row['Tissue_type']
    mlmarker_tissues = metadata_to_mlmarker.get(tissue_type, [])
    
    # Handle case where mlmarker_tissues is an empty list
    if not mlmarker_tissues:
        return "no match"
    
    # Drop columns that are not prediction scores
    if n > 2:
        to_drop = ['Tissue_type', 'In_Top_1']
    else:
        to_drop = ['Tissue_type']

    # Get top N tissues by prediction score
    top_n_tissues = row.drop(to_drop).apply(pd.to_numeric, errors='coerce').nlargest(n).index.tolist()
    
    # Check if any expected tissue is in the top N
    return any(tissue in top_n_tissues for tissue in mlmarker_tissues)

# Process the subset of the DataFrame
subset = df_prediction_full.iloc[:, :35]
subset.reset_index(inplace=True)

# Apply the modified function to the DataFrame
subset['In_Top_1'] = subset.apply(lambda row: check_top_n(row, 1), axis=1)
subset['In_Top_3'] = subset.apply(lambda row: check_top_n(row, 3), axis=1)


In [ ]:
print(subset['In_Top_1'].value_counts())
print(subset['In_Top_3'].value_counts())


In [ ]:
# Create a DataFrame with Tissue_type as index
top3_true_counts = subset[subset['In_Top_3'] == True]['Tissue_type'].value_counts()
top3_false_counts = subset[subset['In_Top_3'] == False]['Tissue_type'].value_counts()

df_counts = pd.DataFrame({'Top 3 True': top3_true_counts, 'Top 3 False': top3_false_counts})
df_counts.index.name = 'Tissue_type'

df_counts['Total'] = subset['Tissue_type'].value_counts()
df_counts['%True'] = df_counts['Top 3 True'] / df_counts['Total']*100
df_counts['%False'] = df_counts['Top 3 False'] / df_counts['Total']*100

df_counts.fillna(0, inplace=True)
df_counts

In [ ]:
#grouped barplot of df_counts, per tissue one bar with True, one bar with False
df_counts[['%True', '%False']].plot(kind='bar', stacked=True, figsize=(10, 6), color=['seagreen', 'lightgrey'])


Two study cases: Brain and Liver, which proteins are responsible for its classification errors?

In [ ]:
df_prediction_full

In [ ]:
df.reset_index(inplace=True)

In [ ]:
livers = df_prediction_full[df_prediction_full.index  == 'Liver'][['Project_Identifier', 'Liver']]
true_livers = livers[livers['Liver'] > 0].Project_Identifier.tolist()
false_livers = livers[livers['Liver'] == 0].Project_Identifier.tolist()

true_livers_proteins = []
for project in range(len(true_livers)):
    true_livers_proteins.append(set(df[['index', true_livers[project]]].dropna()['index'].str.split(';').apply(lambda x: x[0]).tolist()))

false_livers_proteins = []
for project in range(len(false_livers)):
    false_livers_proteins.append(set(df[['index', false_livers[project]]].dropna()['index'].str.split(';').apply(lambda x: x[0]).tolist()))

print(true_livers, false_livers)

#flatten false_livers_proteins
false_livers_proteins = [item for sublist in false_livers_proteins for item in sublist]
true_livers_proteins = [item for sublist in true_livers_proteins for item in sublist]

false_livers_proteins = set(false_livers_proteins).intersection(MLMarker_features)
true_livers_proteins = set(true_livers_proteins).intersection(MLMarker_features)

#venndiagram
from matplotlib_venn import venn2
venn2([set(true_livers_proteins), set(false_livers_proteins)], set_labels=('True', 'False'))
plt.title('Proteins in liver samples')
plt.show()

In [ ]:
#what are the 83 proteins taht are unique to true liver samples
unique_true_livers_proteins = set(true_livers_proteins) - set(false_livers_proteins)
unique_false_livers_proteins = set(false_livers_proteins) - set(true_livers_proteins)
len(unique_true_livers_proteins), len(unique_false_livers_proteins)

In [ ]:
from bioservices import UniProt
u = UniProt()
unique_true_livers_proteins = list(unique_true_livers_proteins)
chunk_size = 10
chunks = [unique_true_livers_proteins[i:i+chunk_size] for i in range(0, len(unique_true_livers_proteins), chunk_size)]

prot_info = pd.DataFrame()
for chunk in chunks:
    chunk_info = u.get_df(list(chunk))
    prot_info = pd.concat([prot_info, chunk_info])
prot_info = prot_info[prot_info['Entry'].isin(unique_true_livers_proteins)] 
prot_info

liver_count = prot_info['Tissue specificity'].str.contains('liver', case=False).sum()
not_annotated  = prot_info['Tissue specificity'].isna().sum()
other = len(prot_info) - liver_count - not_annotated
liver_count, not_annotated, other



In [ ]:
from bioservices import UniProt
u = UniProt()
unique_false_livers_proteins = list(unique_false_livers_proteins)
chunk_size = 10
chunks = [unique_false_livers_proteins[i:i+chunk_size] for i in range(0, len(unique_false_livers_proteins), chunk_size)]

prot_info_false = pd.DataFrame()
for chunk in chunks:
    chunk_info = u.get_df(list(chunk))
    prot_info_false = pd.concat([prot_info_false, chunk_info])
prot_info_false = prot_info_false[prot_info_false['Entry'].isin(unique_false_livers_proteins)] 
prot_info_false

liver_count_false = prot_info['Tissue specificity'].str.contains('liver', case=False).sum()
not_annotated  = prot_info['Tissue specificity'].isna().sum()
other = len(prot_info) - liver_count_false - not_annotated
liver_count_false, not_annotated, other


In [ ]:
hpa_liverspecific = ['Q13103', 'P02765', 'P36980', 'P11226', 'P00740', 'Q9BXR6', 'P04217', 'P01008', 'P02652', 'P00734', 'Q14973', 'Q02985', 'P02790', 'P19827', 'P05543', 'Q9Y6L6', 'Q9UJM8', 'P02749', 'P19823', 'Q92496', 'P55056', 'Q96N76', 'P02671', 'O15245', 'P02675', 'P05177', 'P10632', 'Q6Q0C1', 'Q13790', 'P02679', 'P21549', 'P00748', 'P05181', 'P07327', 'P19652', 'Q63ZE4', 'P07357', 'P02748', 'Q03591', 'Q6Q788', 'P05160', 'P02763', 'P04196', 'P55103', 'P11509', 'O00624', 'Q02325', 'Q8N8V2', 'O75452', 'P36537', 'Q9BQQ7', 'P02768', 'Q9UK05', 'P02655', 'P07358', 'P35542', 'P22680', 'Q9UK55', 'Q9NSA1', 'P06133', 'Q06033', 'P27169', 'P08709', 'P58166', 'Q86U17', 'P00738', 'Q96IY4', 'P51857', 'P02741', 'P02743', 'P20853', 'P02760', 'P22310', 'P08519', 'Q7Z5P4', 'P04004', 'Q969I6', 'Q04756', 'O00187', 'P15169', 'O14960', 'P03950', 'P07307', 'P48775', 'O15467', 'Q96PD5', 'P01009', 'P35218', 'P07306', 'Q14032', 'P31513', 'P04070', 'P20813', 'Q14624', 'P49638', 'Q8WYK0', 'P24462', 'P20742', 'P08319', 'P01031', 'P54840', 'Q8IVM8', 'P11150', 'Q9UNU6', 'P03952', 'O43174', 'P10635', 'Q9UGM5', 'O14756', 'P02656', 'Q9Y2P5', 'P18428', 'Q969E1', 'Q00266', 'O60809', 'P20132', 'P17735', 'Q9Y2D1', 'P02750', 'P00751', 'P11712', 'Q15166', 'P0DJI8', 'P0DJI9', 'P07438', 'P02753', 'Q9NPD5', 'P07360', 'P08185', 'P17516', 'A6NKF2', 'P02774', 'Q14994', 'P21439', 'Q9HB55', 'P08697', 'P22792', 'Q8NEV9', 'Q9UP52', 'P28332', 'Q8WWZ8', 'P35503', 'O95342', 'Q9Y5C1', 'P06276', 'P20851', 'P28845', 'P11168', 'P43652', 'Q6UXH0', 'Q14520', 'Q6NUN0', 'P04003', 'P13671', 'Q8IVS8', 'P35858', 'Q4G0N4', 'P00747', 'A0A0G2JMD5', 'Q86YT5', 'P02771', 'O43315', 'Q9UI32', 'Q8IU80', 'P02654', 'P01024', 'Q4G0S7', 'P23141', 'Q9Y694', 'A6NLP5', 'Q6ZNB7', 'Q8NI99', 'Q9P126', 'Q16696', 'Q02127', 'Q9H8M9', 'P22891', 'Q9UBR1', 'Q5TCH4', 'Q7Z4W1', 'P46952', 'P54868', 'P01042', 'Q9NP71', 'Q9NQ94', 'P01019', 'P04424', 'P06681', 'P08603', 'Q9Y6Z7', 'P33261', 'Q9H1X3'', ', 'P01588', 'Q08830', 'P49326', 'O95954', 'A6NH11', 'P00739', 'P08833', 'Q9NPH3', 'Q8N371', 'Q8TDS5', 'Q6T423', 'Q9BWD1', 'P02647', 'Q6NXE6', 'Q02338', 'P09871', 'P05156', 'P31327', 'P19875', 'Q9NUI1', 'P30047', 'P23378', 'P80108', 'O43708', 'P81172', 'P32754', 'Q9H2F3', 'O15503', 'P11586', 'P49914', 'Q8WWR8', 'Q9UJX0', 'Q8NBP7', 'Q9BY49', 'Q9NST1', 'P29622', 'P05546', 'P04278', 'Q9UJS0', 'H3BR10', 'Q06520', 'P40225', 'Q96MV1', 'Q3YBM2', 'Q8N5A5', 'P45954', 'Q99424', 'O95445', 'P05089', 'P00450', 'Q9NYL5', 'P08684', 'Q8N4W6', 'P03951', 'Q15485', 'A2VDF0', 'P38435', 'Q6ZVE7', 'P48357', 'Q9H400', 'Q86UD1', 'Q92786', 'P34096']


print(f"Amount of HPA specific proteins in True: {}, and uniprot_specific proteins {};    Amount of HPA in False: {} and uniprot {}")
set(unique_true_livers_proteins).intersection(set(hpa_liverspecific)), set(unique_false_livers_proteins).intersection(set(hpa_liverspecific))


In [ ]:
prot_info

In [ ]:
from bioservices import UniProt
u = UniProt()
unique_false_livers_proteins = list(unique_false_livers_proteins)
chunk_size = 10
chunks = [unique_false_livers_proteins[i:i+chunk_size] for i in range(0, len(unique_false_livers_proteins), chunk_size)]

prot_info = pd.DataFrame()
for chunk in chunks:
    chunk_info = u.get_df(list(chunk))
    prot_info = pd.concat([prot_info, chunk_info])
prot_info = prot_info[prot_info['Entry'].isin(unique_false_livers_proteins)] 
prot_info

liver_count = prot_info['Tissue specificity'].str.contains('liver', case=False).sum()
not_annotated  = prot_info['Tissue specificity'].isna().sum()
other = len(prot_info) - liver_count - not_annotated
liver_count, not_annotated, other



In [ ]:

from gprofiler import GProfiler
import plotly.graph_objects as go
# Initialize g:Profiler
gp = GProfiler(return_dataframe=True)

# Dictionary to store GO terms and p-values for each tissue
go_dict = {}


# Perform GO enrichment
results = gp.profile(organism='hsapiens', query=unique_true_livers_proteins, sources=['GO:BP', 'GO:MF', 'GO:CC', 'HPA'])
results = results[results['p_value']< 0.05]
# Store results in the dictionary: {tissue: {GO_term: p-value}}
results

In [ ]:

from gprofiler import GProfiler
import plotly.graph_objects as go
# Initialize g:Profiler
gp = GProfiler(return_dataframe=True)

# Dictionary to store GO terms and p-values for each tissue
go_dict = {}


# Perform GO enrichment
results = gp.profile(organism='hsapiens', query=false_livers_proteins, sources=['GO:BP', 'GO:MF', 'GO:CC', 'HPA'])
results = results[results['p_value']< 0.05]
# Store results in the dictionary: {tissue: {GO_term: p-value}}
results

Central nervous system

In [ ]:
livers = df_prediction_full[df_prediction_full.index  == 'Central Nervous System'][['Project_Identifier', 'Brain']]
true_livers = livers[livers['Brain'] > 0].Project_Identifier.tolist()
false_livers = livers[livers['Brain'] == 0].Project_Identifier.tolist()

true_livers_proteins = []
for project in range(len(true_livers)):
    true_livers_proteins.append(set(df[['index', true_livers[project]]].dropna()['index'].str.split(';').apply(lambda x: x[0]).tolist()))

false_livers_proteins = []
for project in range(len(false_livers)):
    false_livers_proteins.append(set(df[['index', false_livers[project]]].dropna()['index'].str.split(';').apply(lambda x: x[0]).tolist()))

print(true_livers)
print(false_livers)

#flatten false_livers_proteins
false_livers_proteins = [item for sublist in false_livers_proteins for item in sublist]
true_livers_proteins = [item for sublist in true_livers_proteins for item in sublist]

#venndiagram
from matplotlib_venn import venn2
venn2([set(true_livers_proteins), set(false_livers_proteins)], set_labels=('True', 'False'))
plt.title('Proteins in brain samples')
plt.show()

## How many not brain cell lines are predicted as brain?

In [ ]:
df_prediction_full

In [ ]:
sns.boxplot(df_prediction_full[df_prediction_full.index=='Lung']['number_of_proteins'])


In [ ]:
sns.boxplot(df_prediction_full[df_prediction_full.index!='Lung']['number_of_proteins'], color='red')

In [ ]:
df_prediction_full[df_prediction_full.index=='Lung'].iloc[:,:35].sum(axis=0).sort_values(ascending=False).plot(kind='bar', figsize=(10, 6), color='darkblue', title='Classification of lung cell line samples')
